In [ ]:
#import argparse
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
#from get_embedding_function import get_embedding_function
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

argparse: Facilita a criação de comandos de linha de comando e permite que o código receba parâmetros diretamente via CLI.

Chroma: Um banco de dados vetorial para armazenar e recuperar embeddings de texto. Utiliza o diretório persistente CHROMA_PATH.

ChatPromptTemplate: Gerencia prompts para modelos de linguagem.

Ollama: Interface para usar o modelo de linguagem Ollama, que responde a perguntas com base em prompts.

PyPDFDirectoryLoader: Carrega documentos PDF de um diretório.

get_embedding_function: Importa uma função personalizada que cria embeddings de texto.

RecursiveCharacterTextSplitter: Divide documentos em chunks menores, mantendo uma sobreposição entre eles.

Document: Classe de documentos usada para organizar e manipular chunks. [OK]

In [ ]:
CHROMA_PATH = 'chroma_directory'
DATA_PATH = "pdfs"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

CHROMA_PATH: Diretório onde o banco de dados Chroma será armazenado.
DATA_PATH: Diretório de onde os PDFs serão carregados.
PROMPT_TEMPLATE: Um template de prompt que será usado para solicitar respostas do modelo, fornecendo contexto e uma pergunta. [OK]

In [ ]:
def load_and_split_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    print("ETAPA 01 " + str(document_loader))
    
    # Carrega os documentos do diretório
    documents = document_loader.load()
    
    # Verifica e exibe quais arquivos foram encontrados
    if not documents:
        print("Nenhum arquivo PDF encontrado no diretório especificado.")
    else:
        print(f"ETAPA 02: {len(documents)} arquivos encontrados:")
        for doc in documents:
            print(f" - Arquivo: {doc.metadata['source']}")
    
    # Divide os documentos em chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=800,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    print("ETAPA 03: " + str(chunks))
    
    return chunks

# Chame a função para verificar
load_and_split_documents()


Objetivo: Carrega todos os documentos PDF do diretório especificado e os divide em chunks.
document_loader: Inicializa o carregador de PDFs.
documents: Carrega todos os documentos do diretório DATA_PATH.
text_splitter: Configura o splitter para dividir documentos em chunks de até 2000 caracteres com uma sobreposição de 800 caracteres (Overlap)
chunks: Chunks divididos são retornados. [OK]

In [ ]:
def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0

    for i, chunk in enumerate(chunks):
        # Extrai as informações do chunk
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # Debug: Exibe o estado atual
        print(f"Iteração {i + 1}:")
        print(f"  Fonte: {source}")
        print(f"  Página: {page}")
        print(f"  ID da Página Atual: {current_page_id}")
        print(f"  Último ID da Página: {last_page_id}")

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calcula o ID do chunk
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        chunk.metadata["id"] = chunk_id

        # Debug: Exibe o ID calculado e atribuído
        print(f"  ID do Chunk Calculado: {chunk_id}")
        print(f"  Chunk Metadata Atualizada: {chunk.metadata}")

        # Atualiza o último ID da página
        last_page_id = current_page_id

    return chunks

# Exemplo de chamada da função para verificação
# Suponha que você tenha uma lista de chunks para passar para a função
chunks = [
    Document(metadata={"source": "document1.pdf", "page": 1}, page_content="Conteúdo da página 1"),
    Document(metadata={"source": "document1.pdf", "page": 1}, page_content="Conteúdo da página 1 - continuação"),
    Document(metadata={"source": "document1.pdf", "page": 2}, page_content="Conteúdo da página 2"),
    Document(metadata={"source": "document2.pdf", "page": 1}, page_content="Conteúdo da página 1 do documento 2")
]

# Chame a função para verificar
calculate_chunk_ids(chunks)


Objetivo: Calcula e atribui IDs únicos para cada chunk de documento, baseados na origem (nome do documento) e página.
last_page_id: Armazena o ID da última página processada.
current_chunk_index: Índice do chunk atual na mesma página.
chunk_id: ID único gerado para cada chunk, combinando nome do documento, número da página e índice do chunk.

In [ ]:
def add_to_chroma(chunks):
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    chunks_with_ids = calculate_chunk_ids(chunks)

    existing_items = db.get(include=[])
    existing_ids = set(existing_items["ids"])

    new_chunks = [chunk for chunk in chunks_with_ids if chunk.metadata["id"] not in existing_ids]

    if new_chunks:
        db.add_documents(new_chunks, ids=[chunk.metadata["id"] for chunk in new_chunks])
        db.persist()
        print(f"Added {len(new_chunks)} new documents to Chroma.")
    else:
        print("No new documents to add.")

    # Verificação adicional para garantir que os documentos foram adicionados corretamente
    existing_items = db.get(include=[])
    print(f"Existing documents in Chroma: {len(existing_items['ids'])}")

add_to_chroma(chunks)


Objetivo: Adiciona chunks ao banco de dados Chroma, garantindo que apenas chunks novos sejam adicionados.
embedding_function: Obtém a função de embedding personalizada.
db: Inicializa a conexão com o banco de dados Chroma usando a função de embedding.
chunks_with_ids: Chunks com IDs calculados são preparados.
existing_items: Recupera documentos existentes no Chroma.
new_chunks: Filtra chunks novos que não estão no Chroma.
db.add_documents: Adiciona os novos chunks ao banco de dados.
db.persist: Persiste as mudanças no banco de dados Chroma.

In [ ]:

from langchain_community.embeddings.ollama import OllamaEmbeddings

def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

Importação: Importa a classe OllamaEmbeddings para usar embeddings de texto.
Função get_embedding_function: Cria e retorna uma instância do OllamaEmbeddings com o modelo nomic-embed-text.

Uso: A função fornece um objeto que pode ser usado para gerar representações vetoriais (embeddings) para textos.

In [ ]:
def query_rag(query_text):
    print("Initializing embedding function...")  # Debug: Inicializando função de embeddings
    embedding_function = get_embedding_function()
    print("Embedding function initialized.")  # Debug: Função de embeddings inicializada

    print(f"Loading Chroma database from {CHROMA_PATH}...")  # Debug: Carregando Chroma DB
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
    print("Chroma database loaded.")  # Debug: Chroma DB carregado

    print(f"Performing similarity search for query: '{query_text}'...")  # Debug: Realizando busca por similaridade
    results = db.similarity_search_with_score(query_text, k=5)
    print(f"Search completed. Number of results: {len(results)}")  # Debug: Busca concluída

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    print("Context extracted from results.")  # Debug: Contexto extraído dos resultados

    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(f"Prompt created: {prompt}")  # Debug: Exibindo prompt criado

    print("Invoking the model...")  # Debug: Invocando o modelo
    # Adaptando para usar o Ollama com o servidor local
    model = Ollama(model="mistral", base_url="http://127.0.0.1:11434")
    response_text = model.invoke(prompt)
    print(f"Model response received: {response_text}")  # Debug: Resposta do modelo recebida

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)  # Exibindo a resposta formatada
    return response_text



Inicializa a função de embeddings usando `OllamaEmbeddings`.
Carrega o banco de dados Chroma.
Realiza uma busca por similaridade no banco de dados usando o texto da consulta.
Extrai o contexto dos resultados da busca.
Cria um prompt para o modelo baseado no contexto e na pergunta.
Invoca o modelo Ollama para obter uma resposta.
Formata e imprime a resposta e as fontes dos documentos relacionados.

In [ ]:

def main():
    query_text = "Quais são algumas das preocupações éticas associadas ao uso de algoritmos de inteligência artificial, conforme descrito no texto?"
    print(f"Query text received: {query_text}")  # Debug: Exibe a query recebida
    query_rag(query_text)

Exibe o texto da consulta recebido, para depuração.
Chama a função query_rag com o texto da consulta para processar a busca e obter uma resposta.

In [ ]:
main()